In [7]:
import pandas as pd
import yaml
from pathlib import Path
from costometer.utils import AnalysisObject
import pingouin as pg
from statsmodels.tools.eval_measures import bic

# Prepare data for looking at first 10 vs last 10 trials in test 

In [8]:
with open(Path("../../../data/inputs/yamls/experiments/methods_main.yaml"), "rb") as f:
    data = yaml.safe_load(f)

data['trials_per_block'] = {'test_begin':10, 'test_end':10}
with open(Path(f"../../../data/inputs/yamls/experiments/methods_main_transfer.yaml"), "w") as f:
    yaml.dump(data, f)

In [9]:
mouselab_data = pd.read_csv(Path("../../../data/processed/methods_main/mouselab-mdp.csv"), index_col=0)

ranges_to_extract = {'none':"range(0,20)", 'test_begin': "range(20,30)", "test_end": "range(30,40)"}
    
# TODO: refactor out
def expand_range_dictionary(input_dictionary):
    trial_to_block = {}
    for block, trial_range in input_dictionary.items():
        if isinstance(trial_range, str):
            for trial_index in eval(trial_range):
                trial_to_block[trial_index] = block
        else:
            trial_to_block[block] = expand_range_dictionary(trial_range)
    return trial_to_block

trial_to_block = expand_range_dictionary(ranges_to_extract)

mouselab_data["block"] = mouselab_data.apply(lambda row: trial_to_block[row["run"]][row["trial_index"]] if row["run"] in trial_to_block else trial_to_block[row["trial_index"]], axis=1)

mouselab_data = mouselab_data[mouselab_data["block"].isin(["test_begin", "test_end"])]

Path(f"../../../data/processed/methods_main_transfer/").mkdir(exist_ok=True, parents=True)
mouselab_data.to_csv(Path(f"../../../data/processed/methods_main_transfer/mouselab-mdp.csv"), index=False)

In [10]:
mouselab_data.groupby(["pid", "block"]).count()

success  trial_type  time_elapsed  internal_node_id  \
pid  block                                                             
1723 test_begin        0          10            10                10   
     test_end          0          10            10                10   
1731 test_begin        0          10            10                10   
     test_end          0          10            10                10   
1737 test_begin        0          10            10                10   
...                  ...         ...           ...               ...   
2372 test_end          0          10            10                10   
2373 test_begin        0          10            10                10   
     test_end          0          10            10                10   
2374 test_begin        0          10            10                10   
     test_end          0          10            10                10   

                 view_history  rt  responses  correct  revealed_states  \
pid  block                                                               
1723 test_begin             0  10          0        0               10   
     test_end               0  10          0        0               10   
1731 test_begin             0  10          0        0               10   
     test_end               0  10          0        0               10   
1737 test_begin             0  10          0        0               10   
...                       ...  ..        ...      ...              ...   
2372 test_end               0  10          0        0               10   
2373 test_begin             0  10          0        0               10   
     test_end               0  10          0        0               10   
2374 test_begin             0  10          0        0               10   
     test_end               0  10          0        0               10   

                 num_clicks_accrued  ...  variance  branching  final_bonus  \
pid  block                           ...                                     
1723 test_begin                  10  ...        10         10           10   
     test_end                    10  ...        10         10           10   
1731 test_begin                  10  ...        10         10           10   
     test_end                    10  ...        10         10           10   
1737 test_begin                  10  ...        10         10           10   
...                             ...  ...       ...        ...          ...   
2372 test_end                    10  ...        10         10           10   
2373 test_begin                  10  ...        10         10           10   
     test_end                    10  ...        10         10           10   
2374 test_begin                  10  ...        10         10           10   
     test_end                    10  ...        10         10           10   

                 displayed_bonus  time_diff  num_nodes  num_early  num_middle  \
pid  block                                                                      
1723 test_begin               10         10         10         10          10   
     test_end                 10         10         10         10          10   
1731 test_begin               10         10         10         10          10   
     test_end                 10         10         10         10          10   
1737 test_begin               10         10         10         10          10   
...                          ...        ...        ...        ...         ...   
2372 test_end                 10         10         10         10          10   
2373 test_begin               10         10         10         10          10   
     test_end                 10         10         10         10          10   
2374 test_begin               10         10         10         10          10   
     test_end                 10         10         10         10          10   

                 num_late  num_clicks  
pid  blo

In [11]:
for file_idx in range(10):
    print(f"condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw param_file=params_full_five_modified{file_idx}")

condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw param_file=params_full_five_modified0
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw param_file=params_full_five_modified1
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw param_file=params_full_five_modified2
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw param_file=params_full_five_modified3
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw param_file=params_full_five_modified4
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main_transfer cost_function=back_di

condor_submit_bid 1 submission_scripts/MPI-IS/05_Combine_Human.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw

condor_submit_bid 1 submission_scripts/MPI-IS/M_01_Get_MAP_File.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw

# Look at training vs test for now

In [ ]:
mouselab_data["num_queries"]=mouselab_data["queries"].apply(lambda queries: len(eval(queries)["click"]["state"]["target"]))

In [ ]:
num_queries_dict = mouselab_data.groupby(["pid"]).sum()["num_queries"].to_dict()

In [ ]:
 mouselab_data.groupby(["block", "pid"], as_index=False).sum()

In [ ]:
analysis_obj = AnalysisObject(
    "MainExperiment",
    irl_path=Path("../../..").resolve(),
    experiment_subdirectory="methods/static",
)

In [ ]:
test_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["test"], include_null=True)
test_fit["Number Trials"] = test_fit["trace_pid"].apply(lambda trace_pid : num_queries_dict[trace_pid])
sum_test = test_fit.groupby(["Model Name", "Number Parameters"]).sum()[["Number Trials", "value"]].reset_index()

sum_test["bic"] = sum_test.apply(
    lambda row: bic(
    llf=row["value"],
    nobs=row["Number Trials"],
    df_modelwc=row["Number Parameters"]
    ),
    axis=1)
    
sum_test.sort_values(by="bic")

In [ ]:
training_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["training"], include_null=True)
training_fit.groupby(["Model Name"]).sum()["bic"].sort_values()

In [ ]:
test_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["test"], include_null=True)
test_fit.groupby(["Model Name"]).sum()["bic"].sort_values()

In [ ]:
test_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["test"], include_null=True)
test_fit["Number Trials"] = test_fit["trace_pid"].apply(lambda trace_pid : num_queries_dict[trace_pid])
sum_test = test_fit.groupby(["Model Name", "Number Parameters"]).sum()[["Number Trials", "value"]].reset_index()

sum_test["bic"] = sum_test.apply(
    lambda row: bic(
    llf=row["value"],
    nobs=row["Number Trials"],
    df_modelwc=row["Number Parameters"]
    ),
    axis=1)
    
sum_test.sort_values(by="bic")

In [ ]:
training_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["training"], include_null=True)
training_fit.groupby(["Model Name"]).sum()["bic"].sort_values()

In [ ]:
test_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["test"], include_null=True)
test_fit.groupby(["Model Name"]).sum()["bic"].sort_values()

In [ ]:
model = "Distance, Effort, Depth and Forward Search Bonus"
cost_functions = ['forw_added_cost', 'given_cost', 'distance_multiplier', 'depth_cost_weight', 'temp']

for cost_function in cost_functions:
    print(cost_function)
    print(pg.wilcoxon(test_fit[test_fit["Model Name"]==model].sort_values("trace_pid")[cost_function],\
    training_fit[training_fit["Model Name"]==model].sort_values("trace_pid")[cost_function]))

In [ ]:
training_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["training"], include_null=True)
training_fit.groupby(["Model Name"]).sum()["bic"].sort_values()

In [ ]:
test_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["test"], include_null=True)
test_fit.groupby(["Model Name"]).sum()["bic"].sort_values()

In [ ]:
model = "Distance, Effort, Depth and Forward Search Bonus"
cost_functions = ['forw_added_cost', 'given_cost', 'distance_multiplier', 'depth_cost_weight', 'temp']

for cost_function in cost_functions:
    print(cost_function)
    print(pg.wilcoxon(test_fit[test_fit["Model Name"]==model].sort_values("trace_pid")[cost_function],\
    training_fit[training_fit["Model Name"]==model].sort_values("trace_pid")[cost_function]))

In [ ]:
model = "All five"
cost_functions = ["back_added_cost",'forw_added_cost', 'given_cost', 'distance_multiplier', 'depth_cost_weight', 'temp']

for cost_function in cost_functions:
    print(cost_function)
    print(pg.wilcoxon(test_fit[test_fit["Model Name"]==model].sort_values("trace_pid")[cost_function],\
    training_fit[training_fit["Model Name"]==model].sort_values("trace_pid")[cost_function]))

# Look at first half vs second half

In [ ]:
analysis_obj = AnalysisObject(
    "MainExperimentTransfer",
    irl_path=Path("../../..").resolve(),
    experiment_subdirectory="methods/static",
)

In [ ]:
import dill as pickle
with open("/Users/valkyrie/git/planning-depth-differences/data/processed/methods_main_transfer/back_dist_depth_eff_forw/mle_and_map_1723.pickle", "rb") as f:
    data = pickle.load(f)

In [ ]:
data["SoftmaxPolicy"].keys()

In [ ]:
data["SoftmaxPolicy"]["test_begin_mle"][()]

In [ ]:
training_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["None"], include_null=True)
training_fit.groupby(["Model Name"]).sum()["bic"].sort_values()

In [ ]:
training_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["test_begin"], include_null=True)
training_fit.groupby(["Model Name"]).sum()["bic"].sort_values()

In [ ]:
test_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["test_begin"], include_null=True)
sum_test = test_fit.groupby(["Model Name", "Number Parameters"]).sum()[["Number Trials", "value"]].reset_index()

sum_test["bic"] = sum_test.apply(
    lambda row: bic(
    llf=row["value"],
    nobs=row["Number Trials"],
    df_modelwc=row["Number Parameters"]
    ),
    axis=1)
    
sum_test.sort_values(by="bic")

In [ ]:
test_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["test_end"], include_null=True)
test_fit.groupby(["Model Name"]).sum()["bic"].sort_values()

In [ ]:
test_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["test_end"], include_null=True)
sum_test = test_fit.groupby(["Model Name", "Number Parameters"]).sum()[["Number Trials", "value"]].reset_index()

sum_test["bic"] = sum_test.apply(
    lambda row: bic(
    llf=row["value"],
    nobs=row["Number Trials"],
    df_modelwc=row["Number Parameters"]
    ),
    axis=1)
    
sum_test.sort_values(by="bic")

In [ ]:
test_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["None"], include_null=True)
sum_test = test_fit.groupby(["Model Name", "Number Parameters"]).sum()[["Number Trials", "value"]].reset_index()

sum_test["bic"] = sum_test.apply(
    lambda row: bic(
    llf=row["value"],
    nobs=row["Number Trials"],
    df_modelwc=row["Number Parameters"]
    ),
    axis=1)
    
sum_test.sort_values(by="bic")

In [ ]:
model = "Distance, Effort, Depth and Forward Search Bonus"
cost_functions = ['forw_added_cost', 'given_cost', 'distance_multiplier', 'depth_cost_weight', 'temp']

for cost_function in cost_functions:
    print(cost_function)
    print(pg.wilcoxon(test_fit[test_fit["Model Name"]==model].sort_values("trace_pid")[cost_function],\
    training_fit[training_fit["Model Name"]==model].sort_values("trace_pid")[cost_function]))

In [ ]:
model = "All five"
cost_functions = ["back_added_cost",'forw_added_cost', 'given_cost', 'distance_multiplier', 'depth_cost_weight', 'temp']

for cost_function in cost_functions:
    print(cost_function)
    print(pg.wilcoxon(test_fit[test_fit["Model Name"]==model].sort_values("trace_pid")[cost_function],\
    training_fit[training_fit["Model Name"]==model].sort_values("trace_pid")[cost_function]))